In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
from torch import nn
from torch.nn import functional
from torchvision import datasets,transforms
from torch.utils.data import DataLoader

BATCHsize=2000
totepochs=20
learningrate=0.005
dev1ce=torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_data=DataLoader(datasets.MNIST(root="./trainingdata/",transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,), (0.5,))]),train=True,download=True),batch_size=BATCHsize,shuffle=True)
test_data=DataLoader(datasets.MNIST(root="./trainingdata/",transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,), (0.5,))]),train=False),batch_size=BATCHsize)
testnum=len(test_data.dataset)
trainnum=len(train_data.dataset)

class network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1=nn.Conv2d(in_channels=1,out_channels=24,kernel_size=5,stride=1,padding=2)
        self.conv2=nn.Conv2d(in_channels=24,out_channels=48,kernel_size=5,stride=1,padding=1)
        self.conv3=nn.Conv2d(in_channels=48,out_channels=64,kernel_size=5,stride=1,padding=2)
        self.fc1=nn.Linear(in_features=576,out_features=64)
        self.fc2=nn.Linear(in_features=64,out_features=10)
    def forward(self,x):
        size=x.size(0)
        op=self.conv1(x)
        op=functional.relu(op)
        op=functional.avg_pool2d(op,2)
        op=self.conv2(op)
        op=functional.relu(op)
        op=functional.avg_pool2d(op,2)
        op=self.conv3(op)
        op=functional.relu(op)
        op=functional.avg_pool2d(op,2)
        op=op.view(size,-1)
        op=self.fc1(op)
        op=torch.sigmoid(op)
        op=self.fc2(op)
        op=torch.sigmoid(op)
        op=functional.log_softmax(op,dim=1)
        return op

model=network().to(dev1ce)
optmzr=torch.optim.Adam(model.parameters(),lr=learningrate)
lossarr_train=[]
errrate_train=[]
lossarr_test=[]
errrate_test=[]
iter_train=[i for i in range(totepochs*len(train_data))]
iter_test=[i for i in range(1,totepochs+1)]

def train(model,device,train_data,optmzr):
    model.train()
    for index,(data,target) in enumerate(train_data):
        data=data.to(device)
        target=target.to(device)
        optmzr.zero_grad()
        predict=model(data)
        loss=functional.nll_loss(predict,target)
        loss.backward()
        optmzr.step()
        lossarr_train.append(loss.item())
        pred=predict.max(1,keepdim=True)[1]
        sucnum=pred.eq(target.view_as(pred)).sum().item()
        err=(BATCHsize-sucnum)/BATCHsize
        errrate_train.append(err)
        print('Accuracy on batch %d: %.3f [%d/%d] (progress of the epoch: %.2f)'%(index+1,(1.-err),sucnum,BATCHsize,((index+1)*BATCHsize)/trainnum))

def test(model,device,test_data):
    model.eval()
    testloss=0
    sucnum=0
    with torch.no_grad():
        for index,(data,target) in enumerate(test_data):
            data=data.to(device)
            target=target.to(device)
            predict=model(data)
            loss=functional.nll_loss(predict,target)
            testloss+=loss.item()
            pred=predict.max(1, keepdim=True)[1]
            sucnum+=pred.eq(target.view_as(pred)).sum().item()
        err=(testnum-sucnum)/testnum
        testloss/=(testnum/BATCHsize)
        lossarr_test.append(testloss)
        errrate_test.append(err)
        print('Accuracy on test data: %.3f [%d/%d]'%((1.-err),sucnum,testnum))

for iter in range(totepochs):
    print('Epoch %d:'%(iter+1))
    train(model,dev1ce,train_data,optmzr)
    test(model,dev1ce,test_data)

x_ticks_epoch=np.arange(1,totepochs+1,1)
x_ticks_iteration=np.arange(0,trainnum*totepochs/BATCHsize,trainnum/BATCHsize)

plt.subplot(2,2,1)
plt.xticks(x_ticks_iteration)
plt.xlabel('iterations')
plt.ylabel('prediction error rate on training data')
plt.plot(iter_train,errrate_train)
plt.text(iter_train[-1],errrate_train[-1],'%.3f'%errrate_train[-1],size=15)
plt.subplot(2,2,2)
plt.xticks(x_ticks_iteration)
plt.xlabel('iterations')
plt.ylabel('NLL-loss on training data')
plt.plot(iter_train,lossarr_train)
plt.text(iter_train[-1],lossarr_train[-1],'%.3f'%lossarr_train[-1],size=15)
plt.subplot(2,2,3)
plt.xticks(x_ticks_epoch)
plt.xlabel('epochs')
plt.ylabel('prediction error rate on test data')
plt.plot(iter_test,errrate_test)
plt.text(iter_test[-1],errrate_test[-1],'%.3f'%errrate_test[-1],size=15)
plt.subplot(2,2,4)
plt.xticks(x_ticks_epoch)
plt.xlabel('epochs')
plt.ylabel('NLL-loss on test data')
plt.plot(iter_test,lossarr_test)
plt.text(iter_test[-1],lossarr_test[-1],'%.3f'%lossarr_test[-1],size=15)
plt.show()